 # Todas las cuestiones relativas al preprocesado figuran en este Collab

#### **Recordatorio**: aunque todo este preprocesado se haya hecho en grupo, se recuerda que la redacción ha de ser personalizada.

---
---

# Length Features

Aquí se recogen algunas variables relacionadas con la longitud de la frase tanto a nivel de token como a nivel de caracter.

In [1]:
import pandas as pd
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/haha_2021_train.csv", sep=',')

In [2]:
text = train_dataframe[['id', 'text', 'is_humor']]
text = text.iloc[0:100,:]

In [3]:
def len_token(tweet):
  tweet_tokens = tweet.replace('\n', ' ').split(' ')
  return len(tweet_tokens)
def n_characters_tweet(tweet):
  tweet = tweet.replace('\n', ' ')
  return len(tweet)

def get_len_token(text):
  return list(map(len_token, text['text'].to_list()))

def get_n_characters_tweet(text):
  return list(map(n_characters_tweet, text['text'].to_list()))

def get_n_characters_tweet(text):
  return list(map(n_characters_tweet, text['text'].to_list()))

text['len_token'] = get_len_token(text)

text['n_char'] = get_n_characters_tweet(text)

text['ratio'] = text['len_token']/text['n_char']

text

,id,text,is_humor,len_token,n_char,ratio
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,19,86,0.220930
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,22,140,0.157143
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,21,112,0.187500
3,tweet4,No se porqué me hago la cabeza deooos,0,8,37,0.216216
4,tweet5,Quisiera saber que hago durante la siesta de l...,0,20,106,0.188679
...,...,...,...,...,...,...
95,tweet96,el pecado que cometió lamujer en el paraíso de...,1,25,140,0.178571
96,tweet97,- El otro día ví a tu suegra y .....\n- ¡Alto ...,1,26,116,0.224138
97,tweet98,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,1,20,97,0.206186
98,tweet99,Escuchar reggaetón \n\nES DE POBRES,0,7,33,0.212121


---
---

# Word Emoji Context Matrix. 🤣 😆 😂 

Descripción: Extracción/guardado de Emojis y depuración del Corpus

Inconvenientes detectados con los emojis:

* 1 Resulta que los emojis hemos podido verificar que countvectorizer los ignora. Es decir si las frases fueran solo emojis el shape de la matriz word context sería (1,1), con valor empty.

* 2 Aun suponiendo que exista alguna alternativa a countvectorizer muchas personas escriben palabras juntas con emojis, e.g.: 'man👨', lo que podría generar un token más del vocabulario que innecesario. Además pueden escribir varios emojis juntos 👨, 👨👨, 👨👨👨, ... generando también un problema en su identificación.

La propuesta ha sido realizar un preproceso como el siguiente. Además, para solucionar el problema de los emoticonos, en lugar de pasárselos a countvectorizer en bruto utilizamos la decodificación a texto de la librería emoji: **emoji.demojize()**

In [4]:
# %%capture
!pip install emoji

In [5]:
import emoji

In [6]:
def extract_emojis(tweet):
  return ''.join(c for c in tweet if c in emoji.UNICODE_EMOJI_ENGLISH)

def get_emojilist(tweet):
  emojistring = extract_emojis(tweet)
  emojilist = [emoji for emoji in emojistring]
  for emoji in emojilist:
    tweet = tweet.replace(emoji, "")
  return tweet, emojilist

def get_emoji_sentence(tweet):
  tweet, emojilist = get_emojilist(tweet)
  emoji_sentence = emoji.demojize(' '.join(emojilist))
  return tweet, emoji_sentence

def filter(text_df):
  filtered_df = pd.DataFrame(map(get_emoji_sentence, text_df['text'].to_list()))
  filtered_df.rename(columns={0: 'sentences_without_emojis', 1: 'emoji_sentences'}, inplace=True)
  return filtered_df

df_filter_output = filter(text)

df_cleaned_sentences = df_filter_output[['sentences_without_emojis']]
# df_cleaned_sentences = df_filter_output['emoji_sentences']

text['text_v2'] = df_filter_output['sentences_without_emojis'].to_list()

emoji_sentences = df_filter_output['emoji_sentences']

text['text_emojis'] = emoji_sentences.to_list()


def get_emoji_vocab(emoji_sentence):
  analyzer = CountVectorizer(binary=False, analyzer='word', # stop_words=more_stop_words,
                              ngram_range=(1, 1)).build_analyzer()
  return (emoji for emoji in analyzer(emoji_sentence))

In [7]:
from sklearn.feature_extraction.text import  CountVectorizer
cv_emoji = CountVectorizer(analyzer=get_emoji_vocab)

In [8]:
try:
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
# Just if there is no emoji
except ValueError:
  emoji_sentences.loc[0,0] = emoji.demojize("🆘")
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
  """emoji_sentences_test = emoji_sentences"""
  """emoji_sentences_test.loc[0,0] = "🆘🆘🆘🆘🆘🆘🆘🆘"""
  """text['text_emojis'] = emoji_sentences_test.to_list()"""
# cv_emoji.get_feature_names()

In [9]:
cv_emoji.get_feature_names()

['face_with_tears_of_joy',
 'light_skin_tone',
 'loudly_crying_face',
 'loudspeaker',
 'male_sign',
 'medium',
 'person_shrugging']

In [10]:
text

,id,text,is_humor,len_token,n_char,ratio,text_v2,text_emojis
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,19,86,0.220930,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,22,140,0.157143,"—Vamos Luke desenfunda tu sable, demuestra tu ...",
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,21,112,0.187500,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",
3,tweet4,No se porqué me hago la cabeza deooos,0,8,37,0.216216,No se porqué me hago la cabeza deooos,
4,tweet5,Quisiera saber que hago durante la siesta de l...,0,20,106,0.188679,Quisiera saber que hago durante la siesta de l...,
...,...,...,...,...,...,...,...,...
95,tweet96,el pecado que cometió lamujer en el paraíso de...,1,25,140,0.178571,el pecado que cometió lamujer en el paraíso de...,
96,tweet97,- El otro día ví a tu suegra y .....\n- ¡Alto ...,1,26,116,0.224138,- El otro día ví a tu suegra y .....\n- ¡Alto ...,
97,tweet98,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,1,20,97,0.206186,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,
98,tweet99,Escuchar reggaetón \n\nES DE POBRES,0,7,33,0.212121,Escuchar reggaetón \n\nES DE POBRES,


---
---


# Detector de cansinos y corrección de repeticiones innecesarias

En twitter hay mucho "pesao" con el pulgar "cansao" y lo dejan mucho tiempo en el móooooooooovil pulsando una letraaaaa. A todos estos, los consideramos unos pesaos y lo vamos a tener en cuenta. No quieren trabajar.

¿Por qué corregiremos también a los pesaos? Porque spellchecker no puede, es fácil ver esto con un ejemplo en código. 

In [11]:
def is_cansino(tweet, tolerancia:int=2):
  # Consideramos pesados a todos aquellos
  # que repitan 3 veces una letra
  """
  Esta función identifica a los pesaos
  Args:
    tolerancia (int): tolerancia al pesao
  """
  l1 = ""
  rep = 0
  letra = ""
  tweet_aux = tweet + '<eos>'
  for index, letra in enumerate(tweet_aux):
      if letra == l1:
          rep += 1
      elif rep >= tolerancia:
          return l1, rep, tolerancia, 1*True
      else:
        rep = 0
        l1 = letra
  return letra, 0, tolerancia, 1*False

def get_cansinos(corpus):
  L = list(map(is_cansino, corpus))
  return [ret[3] for ret in L]

###################################Ç
##### Correccion repeticiones #####
###################################

# Esto es beta solo corrige la primera
# tanda de repeticiones se puede mejorar

def tweet_corrector(tweet):
    letra, rep, tolerancia, bool_ = is_cansino(tweet)
    if bool_ == 1:
        tweet = tweet.replace(rep*letra, '')
        tweet = tweet_corrector(tweet)
    return tweet

def corpus_corrector(corpus):
    L = list(map(tweet_corrector, corpus))
    return L

In [12]:
"""
text.loc[7,'text_v2'] = 'Meeee aaaaabuuuuurroooooo'
text
"""

"\ntext.loc[7,'text_v2'] = 'Meeee aaaaabuuuuurroooooo'\ntext\n"

In [13]:
text['detected_text_cansinos'] = get_cansinos(text['text_v2'].to_list())
text['detected_emojis_cansinos'] = get_cansinos(text['text_emojis'].to_list())
text['detected_cansinos'] = (text['detected_text_cansinos'] + text['detected_emojis_cansinos']) % 2
# text['detected_emojis_cansinos] | text['detected_text_cansinos'] 

text['text_v3'] = corpus_corrector(text['text_v2'].to_list())

text

,id,text,is_humor,len_token,n_char,ratio,text_v2,text_emojis,detected_text_cansinos,detected_emojis_cansinos,detected_cansinos,text_v3
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,19,86,0.220930,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,,0,0,0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,22,140,0.157143,"—Vamos Luke desenfunda tu sable, demuestra tu ...",,0,0,0,"—Vamos Luke desenfunda tu sable, demuestra tu ..."
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,21,112,0.187500,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",,1,0,1,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ..."
3,tweet4,No se porqué me hago la cabeza deooos,0,8,37,0.216216,No se porqué me hago la cabeza deooos,,1,0,1,No se porqué me hago la cabeza deos
4,tweet5,Quisiera saber que hago durante la siesta de l...,0,20,106,0.188679,Quisiera saber que hago durante la siesta de l...,,0,0,0,Quisiera saber que hago durante la siesta de l...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,tweet96,el pecado que cometió lamujer en el paraíso de...,1,25,140,0.178571,el pecado que cometió lamujer en el paraíso de...,,0,0,0,el pecado que cometió lamujer en el paraíso de...
96,tweet97,- El otro día ví a tu suegra y .....\n- ¡Alto ...,1,26,116,0.224138,- El otro día ví a tu suegra y .....\n- ¡Alto ...,,1,0,1,- El otro día ví a tu suegra y .\n- ¡Alto ahí!...
97,tweet98,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,1,20,97,0.206186,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,,0,0,0,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...
98,tweet99,Escuchar reggaetón \n\nES DE POBRES,0,7,33,0.212121,Escuchar reggaetón \n\nES DE POBRES,,0,0,0,Escuchar reggaetón \n\nES DE POBRES


---
---

# SpellChecker (Corrector ortográfico)

In [14]:
# SpellChecker creo que no hace falta, pero por si acaso la dejo
# !pip install SpellChecker

In [15]:
# pyspellchecker si hace falta instalarla
# %%capture
!pip install pyspellchecker
from spellchecker import SpellChecker

In [16]:
# Con distance=1 el corrector es más conservador 
spell_check = SpellChecker(language='es', distance=1) # Gracias Yoan
# spell_check.correction("-")
# (9spell_check  = SpellChecker(language='es', distance=100)
# spell_check.correction("chicxs cómo hestais")

In [17]:
string = 'hola'
'a' in string

True

In [18]:
def corrector_ortografico(tweet):
  tweet = tweet.replace('?', ' ? ')
  tweet = tweet.replace('¿', ' ¿ ')
  tweet = tweet.replace('...', ' . ')
  tweet = tweet.replace('.', ' . ')
  tweet = tweet.replace('—', ' . ')
  tweet = tweet.replace('. .', ' . ')
  tweet = tweet.replace('.  .', ' . ')
  tweet = tweet.replace('.   .', ' . ')
  
  tweet = tweet.replace('\n', '').split(' ')
  

  corrected_tweet = [spell_check.correction(token) if token not in ['', '-', '?', '¿', ] else token for token in tweet]

  return ' '.join(corrected_tweet)

def corrector_corpus(corpus):
  return list(map(corrector_ortografico, corpus))

In [19]:
text['text_v4'] = corrector_corpus(text['text_v3'])

In [20]:
text[['text_v3', 'text_v4']]

,text_v3,text_v4
0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental . Bajo . . Medio...
1,"—Vamos Luke desenfunda tu sable, demuestra tu ...",. Vamos Luke desenfunda tu sable demuestra tu...
2,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz..."
3,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos
4,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...
...,...,...
95,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...
96,- El otro día ví a tu suegra y .\n- ¡Alto ahí!...,- El otro día ví a tu suegra y . - alto ahí s...
97,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,chiste Te amor ¿ En serio ? Que casualidad!-...
98,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaeton ES DE POBRES


In [21]:
text['text_v4'].iloc[6]

' . Buenas don pepe  ¿ me vende un litro de leche ?  .  ¿ Entera ?  . no si quiere tómese un vasito pinche viejo abusivo . '

---
---

# Obtención de Stop Words

In [22]:
# %%capture
!pip install stop_words

In [23]:
from stop_words import get_stop_words # Se añade librería para obtener las stop_word de cualquier idioma
!wget "https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt"

# Se obtienen las stop_words en español
import pickle
with open("more_stop_words.txt", "rb") as f:
  list_test = pickle.load(f)

stop_words = get_stop_words("spanish")
more_stop_words = stop_words + list_test
more_stop_words.sort()

--2021-05-23 11:31:10--  https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10258 (10K) [application/octet-stream]
Saving to: ‘more_stop_words.txt.9’

more_stop_words.txt 100%[===================>]  10.02K  --.-KB/s    in 0s      

2021-05-23 11:31:11 (102 MB/s) - ‘more_stop_words.txt.9’ saved [10258/10258]



# ELIMINACIÓN DE IDIOMAS ALEJADOS DEL ESPAÑOL

In [24]:
!pip install spacy_langdetect

In [25]:
# frase de prueba
# text = '¿Hola cómo سمبأسكمق estáis? 종현아생일축하해'

In [26]:
import spacy
from spacy_langdetect import LanguageDetector

# sentence level language detection
dict_ = {}

nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

# Eliminamos palabras koreanas, vietnamitas, japonesas o árabes
L = ['ko', 'vi', 'ja', 'ar']

print("v3: ", text['text_v3'].iloc[6])
print("\n\n v4: ", text['text_v4'].iloc[6])

df_cleaned_sentences_list = df_cleaned_sentences['sentences_without_emojis'].to_list()

df_cleaned_sentences_list = text['text_v4'].to_list()

for index, sentence in enumerate(df_cleaned_sentences_list):
  doc = nlp(sentence)
  for subsent in doc.sents:
    if subsent._.language['language'] in L:
      df_cleaned_sentences_list[index] = sentence.replace(subsent.text,'')
df_cleaned_sentences_list

text['text_v5'] = df_cleaned_sentences_list

text

v3:  —Buenas don Pepe, ¿me vende un litro de leche?
—¿Entera?
—No, si quiere tómese un vasito pinche viejo abusivo.


 v4:   . Buenas don pepe  ¿ me vende un litro de leche ?  .  ¿ Entera ?  . no si quiere tómese un vasito pinche viejo abusivo . 


,id,text,is_humor,len_token,n_char,ratio,text_v2,text_emojis,detected_text_cansinos,detected_emojis_cansinos,detected_cansinos,text_v3,text_v4,text_v5
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,19,86,0.220930,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,,0,0,0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental . Bajo . . Medio...,Niveles de retraso mental . Bajo . . Medio...
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,22,140,0.157143,"—Vamos Luke desenfunda tu sable, demuestra tu ...",,0,0,0,"—Vamos Luke desenfunda tu sable, demuestra tu ...",. Vamos Luke desenfunda tu sable demuestra tu...,. Vamos Luke desenfunda tu sable demuestra tu...
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,21,112,0.187500,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",,1,0,1,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz..."
3,tweet4,No se porqué me hago la cabeza deooos,0,8,37,0.216216,No se porqué me hago la cabeza deooos,,1,0,1,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos
4,tweet5,Quisiera saber que hago durante la siesta de l...,0,20,106,0.188679,Quisiera saber que hago durante la siesta de l...,,0,0,0,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tweet96,el pecado que cometió lamujer en el paraíso de...,1,25,140,0.178571,el pecado que cometió lamujer en el paraíso de...,,0,0,0,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...
96,tweet97,- El otro día ví a tu suegra y .....\n- ¡Alto ...,1,26,116,0.224138,- El otro día ví a tu suegra y .....\n- ¡Alto ...,,1,0,1,- El otro día ví a tu suegra y .\n- ¡Alto ahí!...,- El otro día ví a tu suegra y . - alto ahí s...,- El otro día ví a tu suegra y . - alto ahí s...
97,tweet98,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,1,20,97,0.206186,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,,0,0,0,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,chiste Te amor ¿ En serio ? Que casualidad!-...,chiste Te amor ¿ En serio ? Que casualidad!-...
98,tweet99,Escuchar reggaetón \n\nES DE POBRES,0,7,33,0.212121,Escuchar reggaetón \n\nES DE POBRES,,0,0,0,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaeton ES DE POBRES,Escuchar reggaeton ES DE POBRES


---
---

# CREACION MATRIZ WORD CONTEXT DEL TEXTO DEPURADO (SIN EMOJIS y sin idiomas poco habituales)

In [27]:
from nltk.stem.snowball import SpanishStemmer
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=more_stop_words,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))
    
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(analyzer=spanish_stemmer, stop_words=more_stop_words, lowercase=True)
# tf_idf = TfidfTransformer(smooth_idf=True, use_idf=True)
c_vec_mat = c_vec.fit_transform(text['text_v3'])
# tf_idf_mat = tf_idf.fit_transform(c_vec_mat)
c_vec_mat.shape

(100, 529)

In [28]:
df_c_vec = pd.DataFrame(c_vec_mat.toarray(), columns=c_vec.get_feature_names())
print(df_c_vec)

    20cosasquehacerantesdemor  abaj  abraz  absorb  ...  xdxd  xqu  yot  zon
0                           0     0      0       0  ...     0    0    0    0
1                           0     0      0       0  ...     0    0    0    0
2                           0     0      0       0  ...     0    0    0    0
3                           0     0      0       0  ...     0    0    0    0
4                           0     0      0       0  ...     0    0    0    0
..                        ...   ...    ...     ...  ...   ...  ...  ...  ...
95                          0     0      0       0  ...     0    0    0    0
96                          0     0      0       0  ...     0    0    0    0
97                          0     0      0       0  ...     0    0    0    0
98                          0     0      0       0  ...     0    0    0    0
99                          0     0      0       0  ...     0    0    0    0

[100 rows x 529 columns]


---
---

# Traducción Corpus

# Descripción:

Frente a la adversidad que supone que la gran mayoría de modelos desarrollados en HuggingFace se encuentran en Inglés, se ha optado por traducir el corpus al inglés con el pipeline correspondiente. Así, podemos aprovecharlos para hacer Transfer Learning.

Puesto que la fama de los transformers les antecede, consideramos que el error de los mismos de cara a las traducciones resulta asumible.

In [29]:
!pip install transformers
!pip install sentencepiece

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification, pipeline

In [31]:
# Autotokenizador
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")

# Elección/Descarga del modelo Preentrenado
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

In [32]:
def get_corpus_translation(corpus):
  from transformers import pipeline
  pline = pipeline("translation_es_to_en", model=model, tokenizer=tokenizer)

  def get_tweet_translation(tweet):

    tweet = tweet.lower()

    tweet = tweet.replace('.', ' <.> ')
    tweet = tweet.replace('...', ' . ')
    tweet = tweet.replace(' q ', ' que ')
    tweet = tweet.replace(' k ', ' que ')
    tweet = tweet.replace(' d ', ' de ')

    return pline(tweet)[0]['translation_text'] # punto <point>

  return list(map(get_tweet_translation, corpus['text_v5'].to_list()))

text['en_text'] = get_corpus_translation(text)

def recover_point(tweet):
  return tweet.replace('<.>', ' . ')

def recover_point_corpus(corpus):
  return list(map(recover_point, corpus))

text['en_text'] = recover_point_corpus(text['en_text'])



In [33]:
text = text[['id', 'text', 'text_v2', 'text_v3', 'text_v4', 'text_v5', 'en_text', 'len_token', 'n_char', 'ratio', 'detected_cansinos', 'detected_emojis_cansinos', 'detected_text_cansinos', 'text_emojis', 'is_humor']]

text.replace({'text_emojis': {'': 'no_emojis'}}, inplace= True)

corpus = text.copy()

In [34]:
corpus['text_emojis'][3]

'no_emojis'

In [35]:
corpus.to_csv('cleaned_corpus.csv', index=False, sep = ',')

In [36]:
corpus['text_emojis'].loc[99]

':person_shrugging: :medium-light_skin_tone: :male_sign:'

In [37]:
test_corpus = pd.read_csv('cleaned_corpus.csv', header=0, sep=',')

test_corpus.tail(8)

,id,text,text_v2,text_v3,text_v4,text_v5,en_text,len_token,n_char,ratio,detected_cansinos,detected_emojis_cansinos,detected_text_cansinos,text_emojis,is_humor
92,tweet93,Cuando estoy gritando en mi mente mando mensaj...,Cuando estoy gritando en mi mente mando mensaj...,Cuando estoy gritando en mi mente mando mensaj...,Cuando estoy gritando en mi mente mando mensaj...,Cuando estoy gritando en mi mente mando mensaj...,when I am shouting in my mind I send messages ...,12,72,0.166667,0,0,0,no_emojis,0
93,tweet94,La mujer de tu vida no la vas a encontrar un s...,La mujer de tu vida no la vas a encontrar un s...,La mujer de tu vida no la vas a encontrar un s...,La mujer de tu vida no la vas a encontrar un s...,La mujer de tu vida no la vas a encontrar un s...,the woman of your life you won't find her on a...,20,92,0.217391,0,0,0,no_emojis,0
94,tweet95,Cada vez que estoy pasando por momentos difíci...,Cada vez que estoy pasando por momentos difíci...,Cada vez que estoy pasando por momentos difíci...,Cada vez que estoy pasando por momentos difíci...,Cada vez que estoy pasando por momentos difíci...,Every time I'm going through difficult times I...,24,140,0.171429,0,0,0,no_emojis,1
95,tweet96,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,the sin that the woman committed in paradise h...,25,140,0.178571,0,0,0,no_emojis,1
96,tweet97,- El otro día ví a tu suegra y .....\n- ¡Alto ...,- El otro día ví a tu suegra y .....\n- ¡Alto ...,- El otro día ví a tu suegra y .\n- ¡Alto ahí!...,- El otro día ví a tu suegra y . - alto ahí s...,- El otro día ví a tu suegra y . - alto ahí s...,- the other day I saw your mother-in-law and ...,26,116,0.224138,1,0,1,no_emojis,1
97,tweet98,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,chiste Te amor ¿ En serio ? Que casualidad!-...,chiste Te amor ¿ En serio ? Que casualidad!-...,joke I love you seriously what chance!- Why ? ...,20,97,0.206186,0,0,0,no_emojis,1
98,tweet99,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaeton ES DE POBRES,Escuchar reggaeton ES DE POBRES,listening reggaeton is from the poor,7,33,0.212121,0,0,0,no_emojis,0
99,tweet100,Que pena que cuando me vengas a buscar ya no e...,Que pena que cuando me vengas a buscar ya no e...,Que pena que cuando me vengas a buscar ya no e...,Que pena que cuando me vengas a buscar ya no e...,Que pena que cuando me vengas a buscar ya no e...,"Too bad when you come looking for me, I'm no l...",12,55,0.218182,0,0,0,:person_shrugging: :medium-light_skin_tone: :m...,0


In [38]:
test_corpus['text_emojis'][~test_corpus['text_emojis'].isna()]

0                                             no_emojis
1                                             no_emojis
2                                             no_emojis
3                                             no_emojis
4                                             no_emojis
                            ...                        
95                                            no_emojis
96                                            no_emojis
97                                            no_emojis
98                                            no_emojis
99    :person_shrugging: :medium-light_skin_tone: :m...
Name: text_emojis, Length: 100, dtype: object

In [39]:
test_corpus.iloc[6]['text']

'—Buenas don Pepe, ¿me vende un litro de leche?\n—¿Entera?\n—No, si quiere tómese un vasito pinche viejo abusivo...'

In [40]:
test_corpus.iloc[6]['text_v3']

'—Buenas don Pepe, ¿me vende un litro de leche?\n—¿Entera?\n—No, si quiere tómese un vasito pinche viejo abusivo.'

In [41]:
test_corpus.iloc[6]['text_v4']

' . Buenas don pepe  ¿ me vende un litro de leche ?  .  ¿ Entera ?  . no si quiere tómese un vasito pinche viejo abusivo . '

In [42]:
test_corpus.iloc[6]['text_v5']

' . Buenas don pepe  ¿ me vende un litro de leche ?  .  ¿ Entera ?  . no si quiere tómese un vasito pinche viejo abusivo . '

In [43]:
test_corpus.iloc[6]['en_text']

' .  good don pepe does it sell me a liter of milk ?  .  whole ?  .  not if you want to take a pill old abusive  . '

**I'm bored** antes se traducía como **meeee bored**, así que hemos ganado en calidad.

Generar a partir de text_v2, text_v3 donde se pase text_v2 y se haga el spell_checker

In [44]:
# eliminar nulos

df_train = test_corpus[~test_corpus.text_v5.isna()]
df_train

,id,text,text_v2,text_v3,text_v4,text_v5,en_text,len_token,n_char,ratio,detected_cansinos,detected_emojis_cansinos,detected_text_cansinos,text_emojis,is_humor
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental . Bajo . . Medio...,Niveles de retraso mental . Bajo . . Medio...,mental retardation levels . low . . medi...,19,86,0.220930,0,0,0,no_emojis,1
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...","—Vamos Luke desenfunda tu sable, demuestra tu ...","—Vamos Luke desenfunda tu sable, demuestra tu ...",. Vamos Luke desenfunda tu sable demuestra tu...,. Vamos Luke desenfunda tu sable demuestra tu...,. let's luke unscrew your sabre shows your h...,22,140,0.157143,0,0,0,no_emojis,1
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz...","- Do I offer you anything? , do I water coffee...",21,112,0.187500,1,0,1,no_emojis,1
3,tweet4,No se porqué me hago la cabeza deooos,No se porqué me hago la cabeza deooos,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos,I don't know why I'm doing my head.,8,37,0.216216,1,0,1,no_emojis,0
4,tweet5,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,I want to know what I do during the nap from w...,20,106,0.188679,0,0,0,no_emojis,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tweet96,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,el pecado que cometió lamujer en el paraíso de...,the sin that the woman committed in paradise h...,25,140,0.178571,0,0,0,no_emojis,1
96,tweet97,- El otro día ví a tu suegra y .....\n- ¡Alto ...,- El otro día ví a tu suegra y .....\n- ¡Alto ...,- El otro día ví a tu suegra y .\n- ¡Alto ahí!...,- El otro día ví a tu suegra y . - alto ahí s...,- El otro día ví a tu suegra y . - alto ahí s...,- the other day I saw your mother-in-law and ...,26,116,0.224138,1,0,1,no_emojis,1
97,tweet98,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,#Chiste Te amo\n- ¿En serio? Que casualidad!\n...,chiste Te amor ¿ En serio ? Que casualidad!-...,chiste Te amor ¿ En serio ? Que casualidad!-...,joke I love you seriously what chance!- Why ? ...,20,97,0.206186,0,0,0,no_emojis,1
98,tweet99,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaetón \n\nES DE POBRES,Escuchar reggaeton ES DE POBRES,Escuchar reggaeton ES DE POBRES,listening reggaeton is from the poor,7,33,0.212121,0,0,0,no_emojis,0
